# BloodborneIF: Text based RPG with LLM parsing

### Game State Controller

- Player weapon upgrade level
- Current player health
- Current Player insight
- Current player location
- Objects that can be interacted with in the current location
- Player inventory

In [ ]:
from collections import defaultdict
import io
import sys
import os

print_to_stdout = True
print_to_devnull = False

if print_to_stdout:
    game_io = sys.stdout
elif print_to_devnull:
    game_io = open(os.devnull, 'w')
else:
    game_io = io.StringIO()

def print_game_output(*args, **kwargs):
    """Helper used by the game engine to print text to the player."""
    kwargs['file'] = game_io
    print(*args, **kwargs)


## Core World Objects

We keep the same structure as the Action Castle engine:
- **Location**: a node in the world graph
- **Item**: an object that can be in a location or in the player's inventory


In [11]:
class Location:

    def __init__(self, name, description, insight_level=0):
        self.name = name
        self.description = description
        self.properties = defaultdict(bool)
        self.connections = {}           
        self.travel_descriptions = {}   
        self.items = {}                 
        self.blocks = {}                
        self.has_been_visited = False

        self.insight_level = insight_level

    def set_property(self, property_name, property_bool=True):
        self.properties[property_name] = property_bool

    def get_property(self, property_name):
        return self.properties[property_name]

    def set_insight_level(self, value: int):
        self.insight_level = value

    def get_insight_level(self) -> int:
        return self.insight_level

    def add_connection(self, direction, connected_location, travel_description=""):
        direction = direction.lower()
        self.connections[direction] = connected_location
        self.travel_descriptions[direction] = travel_description

        reverse = {
            'north': 'south', 'south': 'north',
            'east': 'west',   'west': 'east',
            'up': 'down',     'down': 'up',
        }
        if direction in reverse:
            rev = reverse[direction]
            if rev not in connected_location.connections:
                connected_location.connections[rev] = self
                connected_location.travel_descriptions[rev] = ""

    def add_block(self, blocked_direction, block_description, preconditions):
        """Add a puzzle/block to a direction.
        `preconditions` is a list of callables: game -> bool.
        """
        self.blocks[blocked_direction] = (block_description, preconditions)

    def is_blocked(self, direction, game):
        if direction not in self.blocks:
            return False
        block_description, preconditions = self.blocks[direction]
        if all(cond(game) for cond in preconditions):
            return False
        print_game_output(block_description)
        return True

    def add_item(self, item):
        self.items[item.name] = item

    def remove_item(self, item_name):
        return self.items.pop(item_name, None)


class Item:
    def __init__(self, name, description, properties=None):
        self.name = name
        self.description = description
        self.properties = defaultdict(bool)
        if properties:
            for k, v in properties.items():
                self.properties[k] = v

    def set_property(self, prop, value=True):
        self.properties[prop] = value

    def get_property(self, prop):
        return self.properties[prop]


## Game State Tracking

- health and max_health
- insight level
- weapon_level
- inventory
- curr_location
- interactable_objects


In [ ]:
class BloodborneGame:
    def __init__(self, start_at, all_locations, max_health=100, weapon_level=1, starting_insight=0, max_insight=99):
        
        # World graph
        self.curr_location = start_at
        self.curr_location.has_been_visited = True
        self.all_locations = all_locations

        # Player-centric state
        self.max_health = max_health
        self.health = max_health
        self.weapon_level = weapon_level
        self.inventory = {}

        # Insight tracking
        self.max_insight = max_insight
        self.insight = starting_insight

        self.state_history = []

    # ---------- Movement and description ----------
    def describe(self):
        print_game_output(f"You are at: {self.curr_location.name}")
        print_game_output(self.curr_location.description)
        self.describe_exits()
        self.describe_items()
        self.describe_status()

    def describe_exits(self):
        exits = [d.capitalize() for d in self.curr_location.connections.keys()]
        if exits:
            print_game_output("Exits: " + ", ".join(exits))
        else:
            print_game_output("There are no obvious exits.")

    def describe_items(self):
        if not self.curr_location.items:
            print_game_output("You see nothing of note here.")
            return
        print_game_output("You see:")
        for item in self.curr_location.items.values():
            print_game_output(f"  - {item.name}: {item.description}")

    def describe_status(self):
        print_game_output(
            f"Health: {self.health}/{self.max_health}  |  "
            f"Weapon +{self.weapon_level}  |  "
            f"Insight: {self.insight}/{self.max_insight}  "
            f"(Area Insight: {self.curr_location.insight_level})"
        )

    def go(self, direction):
        direction = direction.lower()
        if direction not in self.curr_location.connections:
            print_game_output("You can't go that way.")
            return
        if self.curr_location.is_blocked(direction, self):
            return

        new_loc = self.curr_location.connections[direction]
        travel_desc = self.curr_location.travel_descriptions.get(direction, "")
        if travel_desc:
            print_game_output(travel_desc)
        self.curr_location = new_loc
        self.curr_location.has_been_visited = True
        self.log_state()
        self.describe()

    # ---------- Inventory management ----------
    def is_in_inventory(self, item_name):
        return item_name in self.inventory

    def add_to_inventory(self, item):
        self.inventory[item.name] = item
        print_game_output(f"You take the {item.name}.")
        self.log_state()

    def remove_from_inventory(self, item_name):
        item = self.inventory.pop(item_name, None)
        if item:
            print_game_output(f"You drop the {item.name}.")
            self.curr_location.add_item(item)
            self.log_state()
        return item

    def take(self, item_name):
        item_name = item_name.lower()
        if item_name not in self.curr_location.items:
            print_game_output("There is no such item here.")
            return
        item = self.curr_location.remove_item(item_name)
        self.add_to_inventory(item)

    # ---------- Combat, Health, Weapon Upgrades ----------
    def take_damage(self, amount):
        self.health = max(0, self.health - amount)
        print_game_output(f"You take {amount} damage! Health is now {self.health}.")
        self.log_state()

    def heal(self, amount):
        old = self.health
        self.health = min(self.max_health, self.health + amount)
        restored = self.health - old
        print_game_output(f"You recover {restored} health. Health is now {self.health}.")
        self.log_state()

    def upgrade_weapon(self, levels=1):
        self.weapon_level += levels
        print_game_output(f"Your weapon has been strengthened to +{self.weapon_level}.")
        self.log_state()

    # ---------- Insight ----------
    def gain_insight(self, amount=1):
        old = self.insight
        self.insight = min(self.max_insight, self.insight + amount)
        gained = self.insight - old
        if gained > 0:
            print_game_output(f"You gain {gained} Insight. Insight is now {self.insight}.")
        else:
            print_game_output("Your mind cannot grasp any more Insight.")
        self.log_state()

    def lose_insight(self, amount=1):
        old = self.insight
        self.insight = max(0, self.insight - amount)
        lost = old - self.insight
        if lost > 0:
            print_game_output(f"You lose {lost} Insight. Insight is now {self.insight}.")
        else:
            print_game_output("Your Insight cannot sink any lower.")
        self.log_state()

    # ---------- State snapshot ----------
    def get_state(self):

        return {
            'location': self.curr_location.name,
            'health': self.health,
            'max_health': self.max_health,
            'weapon_level': self.weapon_level,
            'inventory': list(self.inventory.keys()),
            'interactable_objects': list(self.curr_location.items.keys()),
            'insight': self.insight,
            'max_insight': self.max_insight,
            'location_insight': self.curr_location.insight_level,
        }

    def log_state(self):
        self.state_history.append(self.get_state())


## Define Locations for World Map (TEMP)

- **Hunter's Dream** – a safe hub, with a **workshop altar** for upgrading your weapon
- **Central Yharnam Gate** – beasts lurk; you can pick up **blood vials**
- **Cathedral Ward** – tougher enemies but better rewards

In [ ]:
def build_world():
    # Locations
    hunters_dream = Location(
    "Hunter's Dream",
    "A pale, moonlit workshop floating above a forgotten city. A comforting fire burns nearby.",
    insight_level = 1
    )

    central_yharnam = Location(
    "Central Yharnam Gate",
    "Cobblestone streets slick with blood. Lanterns flicker as distant howls echo through the fog.",
    insight_level = 3
    )

    cathedral_ward = Location(
    "Cathedral Ward",
    "Tall spires pierce the night sky. A great cathedral looms overhead, its doors sealed by some unseen force.",
    insight_level = 5
    )

    # Connections
    hunters_dream.add_connection('north', central_yharnam, "You step through the headstone and awaken in Yharnam...")
    central_yharnam.add_connection('east', cathedral_ward, "You push open a heavy iron gate into the Cathedral Ward.")

    # Items
    workshop_altar = Item(
        'workshop altar',
        'An intricate altar where you can channel blood echoes to strengthen your weapon.',
    )
    hunters_dream.add_item(workshop_altar)

    blood_vial = Item(
        'blood vial',
        'A small glass vial of thick, red blood. Restores a portion of health when used.',
    )
    central_yharnam.add_item(blood_vial)

    badge = Item(
        'bishop badge',
        'A strange badge depicting a weeping bishop. It hums faintly with eldritch power.',
    )
    cathedral_ward.add_item(badge)

    all_locations = {
        'hunters_dream': hunters_dream,
        'central_yharnam': central_yharnam,
        'cathedral_ward': cathedral_ward,
    }

    return hunters_dream, all_locations

def use_blood_vial(game: BloodborneGame):
    if not game.is_in_inventory('blood vial'):
        print_game_output("You fumble at your belt, but you have no blood vials left.")
        return
    game.inventory.pop('blood vial')
    game.heal(35)
    print_game_output("You inject the blood and feel warmth surge through your veins.")


def use_workshop_altar(game: BloodborneGame):
    if game.curr_location.name != "Hunter's Dream":
        print_game_output("There is no workshop altar here.")
        return
    if 'workshop altar' not in game.curr_location.items:
        print_game_output("The altar is strangely absent.")
        return
    game.upgrade_weapon(1)
    print_game_output("You channel echoes into steel. Your trick weapon feels heavier, deadlier.")


## Initialize the Game

In [12]:
start_location, all_locations = build_world()
game = BloodborneGame(
    start_location,
    all_locations,
    max_health=100,
    weapon_level=1,
    starting_insight=0,   # start “blind”
    max_insight=99
)

game.log_state()
game.describe()
print("\nInitial game state snapshot:")
print(game.get_state())


You are at: Hunter's Dream
A pale, moonlit workshop floating above a forgotten city. A comforting fire burns nearby.
Exits: North
You see:
  - workshop altar: An intricate altar where you can channel blood echoes to strengthen your weapon.
Health: 100/100  |  Weapon +1  |  Insight: 0/99  (Area Insight: 0)

Initial game state snapshot:
{'location': "Hunter's Dream", 'health': 100, 'max_health': 100, 'weapon_level': 1, 'inventory': [], 'interactable_objects': ['workshop altar'], 'insight': 0, 'max_insight': 99, 'location_insight': 0}


## Example: State Tracking 

In [7]:
# Example scripted playthrough to show state tracking
print("--- Move north to Central Yharnam ---")
game.go('north')
print("Current state:")
print(game.get_state())

print("\n--- Take the blood vial ---")
game.take('blood vial')
print("Current state:")
print(game.get_state())

print("\n--- Take damage from a beast ---")
game.take_damage(45)
print("Current state:")
print(game.get_state())

print("\n--- Use a blood vial to heal ---")
use_blood_vial(game)
print("Current state:")
print(game.get_state())

print("\n--- Return to Hunter's Dream and upgrade weapon ---")
game.go('south')  # back to Hunter's Dream
use_workshop_altar(game)
print("Current state:")
print(game.get_state())

print("\n--- Move to Cathedral Ward ---")
game.go('north')      # to Central Yharnam
game.go('east')       # to Cathedral Ward
print("Current state:")
print(game.get_state())

print("\nState history (first 3 snapshots):")
for i, s in enumerate(game.state_history[:3]):
    print(i, s)


--- Move north to Central Yharnam ---
You step through the headstone and awaken in Yharnam...
You are at: Central Yharnam Gate
Cobblestone streets slick with blood. Lanterns flicker as distant howls echo through the fog.
Exits: South, East
You see:
  - blood vial: A small glass vial of thick, red blood. Restores a portion of health when used.
Health: 100/100  |  Weapon +1
Current state:
{'location': 'Central Yharnam Gate', 'health': 100, 'max_health': 100, 'weapon_level': 1, 'inventory': [], 'interactable_objects': ['blood vial']}

--- Take the blood vial ---
You take the blood vial.
Current state:
{'location': 'Central Yharnam Gate', 'health': 100, 'max_health': 100, 'weapon_level': 1, 'inventory': ['blood vial'], 'interactable_objects': []}

--- Take damage from a beast ---
You take 45 damage! Health is now 55.
Current state:
{'location': 'Central Yharnam Gate', 'health': 55, 'max_health': 100, 'weapon_level': 1, 'inventory': ['blood vial'], 'interactable_objects': []}

--- Use a blo